In [2]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO
import src.tools.download

from src.files import FASTAFile, GBFFFile
from tqdm import tqdm
import os
import json 

%load_ext autoreload
%autoreload 2

In [3]:
# Changing my training dataset strategy. Instead going to use the sequences directly from NCBI. 
genome_metadata_df = pd.read_csv('../data/genome_metadata.csv', index_col=0)
genome_ids = genome_metadata_df[genome_metadata_df.phylum == 'Campylobacterota'].index.values 


In [ ]:
antifam_metadata_df = pd.read_csv('../data/antifam_metadata.csv', index_col=0)

antifam = src.tools.download.AntiFam()
lineages = antifam._get_taxonomy_metadata(antifam_metadata_df.ncbi_taxonomy_id.unique())

AntiFam._get_taxonomy_metadata:  33%|███▎      | 2234/6812 [38:31<1:41:21,  1.33s/it]

In [ ]:
# Want to build the dataset for Campylobacterota, but filter out the "suspect" sequences, i.e. those which are both hypothetical
# and have only ab initio evidence (i.e. no evidence of conservation). 


is_hypothetical = lambda df : df.top_hit_product == 'hypothetical protein'
is_ab_initio = lambda df : df.top_hit_evidence_type == 'ab initio prediction'
is_suspect = lambda df : is_hypothetical(df) & is_ab_initio(df) # This will be False for intergenic sequences. 

for genome_id in genome_ids:
    protein_path = f'../data/ncbi/proteins/{genome_id}_protein.faa'
    gbff_path = f'../data/ncbi/gbffs/{genome_id}_protein.faa'

    gbff_df = GBFFFile(gbff_path).to_df()
    gbff_df = gbff_df[(gbff_df.feature_type == 'CDS') & (~gbff_df.pseudo)].copy()
    protein_df = FASTAFile(path=protein_path).to_df(prodigal_output=False)
    assert len(protein_df) == len(gbff_df), 'Expected the number of non-pseudo CDS entries in the GBFF file to match the number of entries in the FASTA file.'

    suspect_ids = gbff_df.index[is_suspect(gbff_df)]
    protein_path = 